In [1]:
import os
import pandas as pd
import numpy as np
from airflow.hooks.mssql_hook import MsSqlHook
from poseidon.util import general
import requests

conf = general.config


# Database Credentials

#def get_special_events():
"""Gets SQL for special events query"""
#se_query = general.file_to_string('../sql/special_events.sql')
#se_conn = MsSqlHook(mssql_conn_id='special_events_sql')
#df = se_conn.get_pandas_df(se_query)

df = pd.read_csv("/data/temp/special_events.csv", parse_dates=['event_start', 'event_end'])
df.head()




[2016-12-28 18:54:07,922] {__init__.py:36} INFO - Using executor SequentialExecutor
[2016-12-28 18:54:08,004] {driver.py:120} INFO - Generating grammar tables from /usr/lib/python2.7/lib2to3/Grammar.txt
[2016-12-28 18:54:08,032] {driver.py:120} INFO - Generating grammar tables from /usr/lib/python2.7/lib2to3/PatternGrammar.txt


,event_title,event_id,event_subtitle,event_type,event_desc,event_loc,event_start,event_end,exp_attendance,exp_participants,event_host,event_url,event_address
0,2017 Hillcrest Farmers' Market (Sundays),48856,NaN,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-12-31 09:00:00,2017-12-31 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street
1,Golden Hill Certified Farmer's Market,48921,NaN,FARMERS,This farmer's market in Golden Hill offers an ...,B Street between 27th Street & 28th Street,2017-12-30 09:30:00,2017-12-30 13:30:00,500/day,50,Brian's Farmer Markets,www.briansfarmermarkets.com,2700 block of B Street
2,City Heights Farmer's Market (Every Saturday),48774,NaN,FARMERS,This farmer's market in City Heights offers an...,Wightman Street between 43rd Street and Fairmo...,2017-12-30 09:00:00,2017-12-30 13:00:00,500/day,100,San Diego County Farm Bureau,www.sdfarmbureau.org,4300 Wightman Street
3,North Park Thursday Market,48721,NaN,FARMERS,This farmer's market in North Park offers loca...,North Park Way between Granada Avenue and 30th...,2017-12-28 15:00:00,2017-12-28 19:30:00,900,70,North Park Main Street,www.northparkmarket.com,2900 North Park Way
4,UTC Farmer's Market (Every Thursday),48670,NaN,FARMERS,This farmer's market in University City offers...,Southbound Regents Road south of Porte de Mera...,2017-12-28 15:00:00,2017-12-28 19:00:00,250,40,Brian's Farmers' Market,www.briansfarmersmarket.com,7397 Regents Road


In [7]:
df['event_id'] = pd.to_numeric(df['event_id'], errors='coerce', downcast='integer')
df['event_start'] = pd.to_datetime(df['event_start'], format="%Y-%m-%d %H:%M:%S", errors='coerce')
df['event_end'] = pd.to_datetime(df['event_end'], format="%Y-%m-%d %H:%M:%S", errors='coerce')
df

,event_title,event_id,event_subtitle,event_type,event_desc,event_loc,event_start,event_end,exp_attendance,exp_participants,event_host,event_url,event_address
0,2017 Hillcrest Farmers' Market (Sundays),48856,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-12-31 09:00:00,2017-12-31 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street
1,Golden Hill Certified Farmer's Market,48921,None,FARMERS,This farmer's market in Golden Hill offers an ...,B Street between 27th Street & 28th Street,2017-12-30 09:30:00,2017-12-30 13:30:00,500/day,50,Brian's Farmer Markets,www.briansfarmermarkets.com,2700 block of B Street
2,City Heights Farmer's Market (Every Saturday),48774,None,FARMERS,This farmer's market in City Heights offers an...,Wightman Street between 43rd Street and Fairmo...,2017-12-30 09:00:00,2017-12-30 13:00:00,500/day,100,San Diego County Farm Bureau,www.sdfarmbureau.org,4300 Wightman Street
3,North Park Thursday Market,48721,None,FARMERS,This farmer's market in North Park offers loca...,North Park Way between Granada Avenue and 30th...,2017-12-28 15:00:00,2017-12-28 19:30:00,900,70,North Park Main Street,www.northparkmarket.com,2900 North Park Way
4,UTC Farmer's Market (Every Thursday),48670,None,FARMERS,This farmer's market in University City offers...,Southbound Regents Road south of Porte de Mera...,2017-12-28 15:00:00,2017-12-28 19:00:00,250,40,Brian's Farmers' Market,www.briansfarmersmarket.com,7397 Regents Road
5,Ocean Beach Certified Farmer's Market (Wednesd...,48618,None,FARMERS,This farmer's market in Ocean Beach offers an ...,Newport Avenue between Bacon and Cable Streets,2017-12-27 16:00:00,2017-12-27 20:00:00,"1,200",60,Ocean Beach Merchant's Association,www.oceanbeachsandiego.com,4900 Block of Newport Avenue
6,2017 Hillcrest Farmers' Market (Sundays),48855,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-12-24 09:00:00,2017-12-24 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street
7,Golden Hill Certified Farmer's Market,48920,None,FARMERS,This farmer's market in Golden Hill offers an ...,B Street between 27th Street & 28th Street,2017-12-23 09:30:00,2017-12-23 13:30:00,500/day,50,Brian's Farmer Markets,www.briansfarmermarkets.com,2700 block of B Street
8,City Heights Farmer's Market (Every Saturday),48773,None,FARMERS,This farmer's market in City Heights offers an...,Wightman Street between 43rd Street and Fairmo...,2017-12-23 09:00:00,2017-12-23 13:00:00,500/day,100,San Diego County Farm Bureau,www.sdfarmbureau.org,4300 Wightman Street
9,North Park Thursday Market,48720,None,FARMERS,This farmer's market in North Park offers loca...,North Park Way between Granada Avenue and 30th...,2017-12-21 15:00:00,2017-12-21 19:30:00,900,70,North Park Main Street,www.northparkmarket.com,2900 North Park Way


In [8]:
# First let's extract every address and provide a lat long for it:
event_addr = df[['event_address']]
event_addr = event_addr.drop_duplicates(subset='event_address')
event_addr.reset_index(inplace=True, drop=True)
event_addr

,event_address
0,3690 Normal Street
1,2700 block of B Street
2,4300 Wightman Street
3,2900 North Park Way
4,7397 Regents Road
5,4900 Block of Newport Avenue
6,500 West B Street
7,3800 block of Ray Street
8,401 West Cedar Street
9,8575 New Salem Street


In [9]:
# Geocode
from poseidon.util import general
def geocode_row(row):
    return general.geocode_address_esri(row)
        
geo_tuples = event_addr['event_address'].map(geocode_row)
geo_df = pd.DataFrame(geo_tuples.tolist(), columns=['latitude','longitude'])

geo_df




[2016-12-23 19:08:36,983] {general.py:150} INFO - ESRI Geocoding for: 3690 Normal Street
[2016-12-23 19:08:37,019] {connectionpool.py:214} INFO - Starting new HTTP connection (1): gis1.sandag.org
[2016-12-23 19:08:37,340] {general.py:159} INFO - No geocode for: 3690 Normal Street
[2016-12-23 19:08:37,341] {general.py:150} INFO - ESRI Geocoding for: 2700 block of B Street
[2016-12-23 19:08:37,344] {connectionpool.py:214} INFO - Starting new HTTP connection (1): gis1.sandag.org
[2016-12-23 19:08:37,504] {general.py:159} INFO - No geocode for: 2700 block of B Street
[2016-12-23 19:08:37,505] {general.py:150} INFO - ESRI Geocoding for: 4300 Wightman Street
[2016-12-23 19:08:37,507] {connectionpool.py:214} INFO - Starting new HTTP connection (1): gis1.sandag.org
[2016-12-23 19:08:37,680] {general.py:162} INFO - Geocode success for: 4300 Wightman Street
[2016-12-23 19:08:37,681] {general.py:150} INFO - ESRI Geocoding for: 2900 North Park Way
[2016-12-23 19:08:37,684] {connectionpool.py:214} 

,latitude,longitude
0,NaN,NaN
1,NaN,NaN
2,32.747982,-117.101839
3,32.747671,-117.130530
4,32.857294,-117.219793
5,NaN,NaN
6,32.717994,-117.167652
7,NaN,NaN
8,32.805632,-116.957910
9,32.915177,-117.139868


In [10]:
ev_index = pd.merge(event_addr, geo_df, left_index=True, right_index=True)
ev_index

,event_address,latitude,longitude
0,3690 Normal Street,NaN,NaN
1,2700 block of B Street,NaN,NaN
2,4300 Wightman Street,32.747982,-117.101839
3,2900 North Park Way,32.747671,-117.130530
4,7397 Regents Road,32.857294,-117.219793
5,4900 Block of Newport Avenue,NaN,NaN
6,500 West B Street,32.717994,-117.167652
7,3800 block of Ray Street,NaN,NaN
8,401 West Cedar Street,32.805632,-116.957910
9,8575 New Salem Street,32.915177,-117.139868


In [16]:
events_updated = pd.merge(df, ev_index, on='event_address')
events_updated.head()

,event_title,event_id,event_subtitle,event_type,event_desc,event_loc,event_start,event_end,exp_attendance,exp_participants,event_host,event_url,event_address,latitude,longitude
0,2017 Hillcrest Farmers' Market (Sundays),48856,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-12-31 09:00:00,2017-12-31 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,NaN,NaN
1,2017 Hillcrest Farmers' Market (Sundays),48855,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-12-24 09:00:00,2017-12-24 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,NaN,NaN
2,2017 Hillcrest Farmers' Market (Sundays),48854,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-12-17 09:00:00,2017-12-17 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,NaN,NaN
3,2017 Hillcrest Farmers' Market (Sundays),48853,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-12-10 09:00:00,2017-12-10 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,NaN,NaN
4,2017 Hillcrest Farmers' Market (Sundays),48852,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-12-03 09:00:00,2017-12-03 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,NaN,NaN


In [18]:
events_updated.sort_values(by='event_start', ascending=False)
events_updated.head(100)

,event_title,event_id,event_subtitle,event_type,event_desc,event_loc,event_start,event_end,exp_attendance,exp_participants,event_host,event_url,event_address,latitude,longitude
0,2017 Hillcrest Farmers' Market (Sundays),48856,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-12-31 09:00:00,2017-12-31 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,NaN,NaN
1,2017 Hillcrest Farmers' Market (Sundays),48855,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-12-24 09:00:00,2017-12-24 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,NaN,NaN
2,2017 Hillcrest Farmers' Market (Sundays),48854,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-12-17 09:00:00,2017-12-17 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,NaN,NaN
3,2017 Hillcrest Farmers' Market (Sundays),48853,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-12-10 09:00:00,2017-12-10 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,NaN,NaN
4,2017 Hillcrest Farmers' Market (Sundays),48852,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-12-03 09:00:00,2017-12-03 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,NaN,NaN
5,2017 Hillcrest Farmers' Market (Sundays),48851,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-11-26 09:00:00,2017-11-26 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,NaN,NaN
6,2017 Hillcrest Farmers' Market (Sundays),48850,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-11-19 09:00:00,2017-11-19 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,NaN,NaN
7,2017 Hillcrest Farmers' Market (Sundays),48849,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-11-12 09:00:00,2017-11-12 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,NaN,NaN
8,2017 Hillcrest Farmers' Market (Sundays),48848,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-11-05 09:00:00,2017-11-05 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,NaN,NaN
9,2017 Hillcrest Farmers' Market (Sundays),48847,None,FARMERS,This farmer's market in Hillcrest offers an ar...,DMV parking lot at 3690 Normal Street ~ Normal...,2017-10-29 09:00:00,2017-10-29 14:00:00,"2,000",400,Hillcrest Business Association,www.hillcrestfarmersmarket.com,3690 Normal Street,NaN,NaN


In [16]:
# Entries with addresses

df_w_adds = df[pd.notnull(df['event_address'])]

# Entries with no addresses

df_wo_adds = df[pd.isnull(df['event_address'])]
df_wo_adds['latitude'] = np.nan
df_wo_adds['longitude'] = np.nan
df_wo_adds = df_wo_adds[['event_id','event_title',\
                            'event_subtitle','event_desc','event_loc',\
                            'event_type','event_start','event_end','event_host',\
                            'event_url','exp_attendance','exp_participants',\
                            'event_address','latitude','longitude']]

rows = list()



/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
# Iterate through entries with addresses

for index, row in df_w_adds.iterrows():
    event_id = row['event_id']
    event_address = row['event_address']
    event_desc = row['event_desc']
    event_end = row['event_end']
    event_host = row['event_host']
    event_loc = row['event_loc']
    event_start = row['event_start']
    event_subtitle = row['event_subtitle']
    event_title = row['event_title']
    event_type = row['event_type']
    event_url = row['event_url']
    exp_attendance = row['exp_attendance']
    exp_participants = row['exp_participants']
    latitude = 'nan'
    longitude = 'nan'
    
    
    
    row = (event_id,event_title,event_subtitle,event_desc,event_loc,\
        event_type,event_start,event_end,event_host,event_url,\
        exp_attendance,exp_participants,event_address,latitude,longitude)

    rows.append(row)

NameError: name 'rows' is not defined